In [1]:
import os
import tensorflow as tf
import numpy as np
import time
import datetime
import pickle
import random
import h5py
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Add, ReLU, LSTM, ConvLSTM2D
from tensorflow.keras.layers import Conv2D, Conv3D, MaxPooling2D, concatenate, Input, AveragePooling2D, TimeDistributed, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.applications import EfficientNetB0
from config import *

In [2]:
# inputs
batch_size = 1 # this is total batchsize using all GPUs, so make divisible by num_gpus!
l_rate = 0.0001

# training data location
file_name_stub = 'hdf5_dm_test_auto_' # dm_july2021_ aim_july2021_expert_ dm_july2021_expert_
# file_name_stub = 'dm_6nov_aim_' 
folder_name = '..\\raw_data\\' 
starting_num = 1 # lowest file name to use in training
highest_num = 30 # highest file name to use in training 4000, 5500, 190, 45, 10

# whether to save model if training and where
model_name = 'all_55k_sub_drop_'
save_dir = '..\\save_model\\'
SAVE_MODEL = True

# whether to resume training from a previous model
IS_LOAD_WEIGHTS_AND_MODEL=False
weights_name = 'test_model_1'

In [4]:
start_time=time.time()

if IS_LOAD_WEIGHTS_AND_MODEL:
    # pick up training from earlier spot
    print('-- loading model from saved file --')
    model = tp_load_model(save_dir, weights_name)
    # TODO: if .p file exists, load and check if matches current config
    hypers_load = pickle.load(open(save_dir+'/'+weights_name+'.p', 'rb'))
    print(hypers_load)

else:
    # useful tutorial for building, https://keras.io/getting-started/functional-api-guide/
    print('-- building model from scratch --')

    base_model = EfficientNetB0(weights='imagenet',input_shape=(input_shape[1:]),include_top=False,drop_connect_rate=0.2)
    if 'randinit' in model_name:
        print('random initialisation!\n\n')
        base_model = EfficientNetB0(weights=None,input_shape=(input_shape[1:]),include_top=False,drop_connect_rate=0.2)
    base_model.trainable = True

    intermediate_model= Model(inputs=base_model.input, outputs=base_model.layers[1].output)
    intermediate_model.trainable = True

    input_1 = Input(shape=input_shape,name='main_in')
    x = TimeDistributed(intermediate_model)(input_1)


-- building model from scratch --


NotImplementedError: Exception encountered when calling TimeDistributed.call().

[1mLayer Functional should implement `def compute_output_shape(self, input_shape)`.[0m

Arguments received by TimeDistributed.call():
  • args=('<KerasTensor shape=(None, 96, 150, 280, 3), dtype=float32, sparse=None, name=main_in>',)
  • kwargs={'mask': 'None'}